<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [x] Get mean absolute error for the test set.
- [x] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [x] Add your own stretch goal(s) !
- [x] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [x] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [x] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [x] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


## Data Cleaning

In [0]:
df['EASE-MENT'].shape[0] - df['EASE-MENT'].isna().sum()

0

- Removing 'EASE-MENT' because all cells are NaN.
- Removing 'BUILDING_CLASS_CATEGORY' because we're only looking at one of them.
- Removing 'APARTMENT_NUMBER' because it shouldn't play a role.

In [0]:
housing = df[
    (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') &
    (df['SALE_PRICE'] > 100_000) & (df['SALE_PRICE'] < 2_000_000)
].drop(columns=[
    'EASE-MENT', 'APARTMENT_NUMBER', 'BUILDING_CLASS_CATEGORY', 'ADDRESS',
    'BUILDING_CLASS_AT_PRESENT',
])

housing['SALE_DATE'] = housing['SALE_DATE'].apply(lambda date: int(date.split('/')[0]))
housing['YEAR_BUILT'] = housing['YEAR_BUILT'].apply(int)
housing['ZIP_CODE'] = housing['ZIP_CODE'].apply(lambda z: str(int(z)))
housing['RESIDENTIAL_UNITS'] = housing['RESIDENTIAL_UNITS'].apply(int)
housing['COMMERCIAL_UNITS'] = housing['COMMERCIAL_UNITS'].apply(int)
housing['TOTAL_UNITS'] = housing['TOTAL_UNITS'].apply(int)
housing['LAND_SQUARE_FEET'] = housing['LAND_SQUARE_FEET'].str.replace(',', '').apply(int)

## Split Train vs Test
- Using data from January — March 2019 to train. 
- Using data from April 2019 to test.

In [0]:
housing_train = housing[housing['SALE_DATE'].isin((1,2,3))].drop(columns=['SALE_DATE'])
housing_test = housing[housing['SALE_DATE'] == 4].drop(columns=['SALE_DATE'])

## Split Target: Sale Price

In [0]:
target_train = housing_train['SALE_PRICE']
target_test = housing_test['SALE_PRICE']
housing_train = housing_train.drop(columns=['SALE_PRICE'])
housing_test = housing_test.drop(columns=['SALE_PRICE'])

#### Quick Reality Check



In [0]:
print(housing_train.shape)
print(target_train.shape)
print(housing_test.shape)
print(target_test.shape)

(2507, 14)
(2507,)
(644, 14)
(644,)


### One Hot "Cat" Encoding

All five of these seem reasonable categories for encoding.

In [0]:
housing_train.select_dtypes(exclude='number').describe().T

,count,unique,top,freq
BOROUGH,2507,5,4,1204
NEIGHBORHOOD,2507,6,OTHER,2382
TAX_CLASS_AT_PRESENT,2507,2,1,2476
ZIP_CODE,2507,122,10312,115
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919


In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
housing_train = encoder.fit_transform(housing_train)
housing_test = encoder.transform(housing_test)

housing_train

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE_11230,ZIP_CODE_11427,ZIP_CODE_10461,ZIP_CODE_11226,ZIP_CODE_11203,ZIP_CODE_11229,ZIP_CODE_11364,ZIP_CODE_11373,ZIP_CODE_11365,ZIP_CODE_11429,ZIP_CODE_11414,ZIP_CODE_11369,ZIP_CODE_11415,ZIP_CODE_11413,ZIP_CODE_11434,ZIP_CODE_11435,ZIP_CODE_10312,ZIP_CODE_10308,ZIP_CODE_10314,ZIP_CODE_11236,ZIP_CODE_11228,ZIP_CODE_11207,ZIP_CODE_11218,ZIP_CODE_11210,ZIP_CODE_11234,...,ZIP_CODE_11004,ZIP_CODE_11217,ZIP_CODE_11224,ZIP_CODE_10453,ZIP_CODE_11232,ZIP_CODE_10462,ZIP_CODE_11372,ZIP_CODE_10464,ZIP_CODE_10458,ZIP_CODE_11368,ZIP_CODE_11416,ZIP_CODE_11692,ZIP_CODE_10468,ZIP_CODE_11225,ZIP_CODE_11221,ZIP_CODE_11233,ZIP_CODE_10456,ZIP_CODE_11238,ZIP_CODE_10455,ZIP_CODE_10460,ZIP_CODE_11102,ZIP_CODE_10459,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6800,1325.0,1930,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4000,2001.0,1940,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,1,0,4210,19,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,3500,2043.0,1925,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,1,0,5212,69,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4000,2680.0,1899,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,1,0,7930,121,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1710,1872.0,1940,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0,0,0,1,0,1,0,0,0,0,0,1,0,4081,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2400,921.0,1950,1,0,0,0,0,1,0,0,0,0,0,0
18130,0,0,0,1,0,1,0,0,0,0,0,1,0,2373,201,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2450,2128.0,1980,1,0,0,1,0,0,0,0,0,0,0,0
18132,0,0,0,1,0,1,0,0,0,0,0,1,0,1132,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4361,1807.0,2018,1,0,1,0,0,0,0,0,0,0,0,0
18134,0,0,0,1,0,1,0,0,0,0,0,1,0,3395,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,6000,621.0,1930,1,0,0,0,0,1,0,0,0,0,0,0


In [0]:
housing_train.shape

(2507, 155)

> It seems to me that encoding categorical features - but doing so within the same column may make more sense. It seems odd that we divide the 5 boroughs into 5 columns. The relationship of this data seems to lose some meaning this way.

## Feature Selection: SelectKBest

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest

In [0]:
selector = SelectKBest(score_func=f_regression, k=24)
X_train_selected = selector.fit_transform(housing_train, target_train)
X_test_selected = selector.transform(housing_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
print(X_train_selected.shape)

(2507, 24)


In [0]:
names = housing_train.columns
selected_mask = selector.get_support()
selected_names = names[selected_mask]
print('Selected Features:', *selected_names, sep='\n')

Selected Features:
BOROUGH_3
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
BLOCK
ZIP_CODE_11230
ZIP_CODE_11365
ZIP_CODE_11434
ZIP_CODE_11218
ZIP_CODE_11210
ZIP_CODE_11215
ZIP_CODE_11375
ZIP_CODE_11357
ZIP_CODE_11209
ZIP_CODE_10303
ZIP_CODE_11225
COMMERCIAL_UNITS
TOTAL_UNITS
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3


## Ridge Regression with Normalization

In [0]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

In [0]:
# Train the dragon
model = Ridge(normalize=True)
model.fit(X_train_selected, target_train)
y_pred = model.predict(X_test_selected)
# MAE
mae = mean_absolute_error(target_test, y_pred)
print(f"MAE: ${mae:,.2f}")

MAE: $164,489.36


## Pipeline

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [0]:
pipline = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    RandomForestClassifier(n_jobs=-1, random_state=42)
)
pipline.fit(housing_train, target_train)
print(f"Validation: {100*pipline.score(housing_test, target_test):.2f}%")

Validation: 0.62%
